In [233]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

$X = 1$ con probabilidad $p$ y $X = 0$ con probabilidad $1-p$.

Luego,
$$
\begin{align}
H(X) &= -p \log(p) - (1-p) \log(1-p) \\
\end{align}
$$


In [234]:
def entropy(prob: np.array) -> float:
    H = 0
    for i in range(prob.shape[0]):
        if prob[i] != 0:
            H += prob[i] * np.log2(prob[i])
    return -H


In [235]:
# Se lee el archivo CSV
data = pd.read_csv('DatasetHojas.csv', sep=',', header=0)
data_clase1 = data[data['Clase'] == 1]
data_clase2 = data[data['Clase'] == 2]

# Se separan los datos de cada clase en sus respectivas variables
largo_clase1 = data_clase1['Largo']
ancho_clase1 = data_clase1['Ancho']
largo_clase2 = data_clase2['Largo']
ancho_clase2 = data_clase2['Ancho']


In [236]:
n_bins = 5

hist_ancho_clase1 = np.histogram(ancho_clase1, bins=n_bins)
hist_ancho_clase2 = np.histogram(ancho_clase2, bins=n_bins)
hist_largo_clase1 = np.histogram(largo_clase1, bins=n_bins)
hist_largo_clase2 = np.histogram(largo_clase2, bins=n_bins)

prob_ancho_clase1 = hist_ancho_clase1[0] / np.sum(hist_ancho_clase1[0])
prob_ancho_clase2 = hist_ancho_clase2[0] / np.sum(hist_ancho_clase2[0])
prob_largo_clase1 = hist_largo_clase1[0] / np.sum(hist_largo_clase1[0])
prob_largo_clase2 = hist_largo_clase2[0] / np.sum(hist_largo_clase2[0])


print("Entropía de ancho de clase 1: ", entropy(prob_ancho_clase1))
print("Entropía de ancho de clase 2: ", entropy(prob_ancho_clase2))
print("Entropía de largo de clase 1: ", entropy(prob_largo_clase1))
print("Entropía de largo de clase 2: ", entropy(prob_largo_clase2))

Entropía de ancho de clase 1:  1.84155491187079
Entropía de ancho de clase 2:  1.7906927146926774
Entropía de largo de clase 1:  1.7206031643621587
Entropía de largo de clase 2:  1.728481135079078


### Entropía conjunta 
$$ H(X,Y) = - \sum_{x} \sum_{y} \ p(x,y) \log p(x,y) $$


In [237]:
hist_clase1, _, _ = np.histogram2d(largo_clase1, ancho_clase1, bins=5)
joint_prob_clase1 = hist_clase1 / np.sum(hist_clase1)

hist_clase2, _, _ = np.histogram2d(largo_clase2, ancho_clase2, bins=5)
joint_prob_clase2 = hist_clase2 / np.sum(hist_clase2)

def joint_entropy(prob: np.ndarray) -> float:
    """ Calcula la entropía conjunta de una matriz de probabilidades """
    H = 0
    for i in range(prob.shape[0]):
        for j in range(prob.shape[1]):
            if prob[i][j] != 0:
                H += prob[i][j] * np.log2(prob[i][j])
    return -H

print('Entropía conjunta clase 1:',joint_entropy(joint_prob_clase1))
print('Entropía conjunta clase 2:',joint_entropy(joint_prob_clase2))

Entropía conjunta clase 1: 3.0321465454645993
Entropía conjunta clase 2: 2.853608585978931


### Entropía condicional
$$
\begin{align}
H(Y|X) &= \sum_{x} \ p(x) H(Y|X = x) \\
&= - \sum_{x} \ p(x) \sum_{y} \ p(y|x) \log p(y|x) \\
&= - \sum_{x} \sum_{y} \ p(x,y) \log p(y|x) \\
\end{align}
$$

In [238]:
def conditional_distribution(joint_probability: np.ndarray)-> np.ndarray:
    """ Calcula la probabilidad condicional dada la probabilidad conjunta """
    conditional_prob = np.zeros(joint_probability.shape)
    for i in range(joint_probability.shape[0]):
        for j in range(joint_probability.shape[1]):
            conditional_prob[i][j] = joint_probability[i][j] / np.sum(joint_probability[i])
    return conditional_prob

conditional_prob_clase1 = conditional_distribution(joint_prob_clase1)
conditional_prob_clase2 = conditional_distribution(joint_prob_clase2)

def conditional_entropy(conditional_prob: np.ndarray, joint_prob: np.ndarray) -> float:
    """ Calcula la entropía condicional de una matriz de probabilidades """
    H = 0
    for i in range(conditional_prob.shape[0]):
        for j in range(conditional_prob.shape[1]):
            if conditional_prob[i][j] != 0:
                H += joint_prob[i][j] * np.log2(conditional_prob[i][j])
    return -H

print('Entropía condicional clase 1 (A1|L1):',conditional_entropy(conditional_prob_clase1, joint_prob_clase1))
print('Entropía condicional clase 2 (A2|L2):',conditional_entropy(conditional_prob_clase2, joint_prob_clase2))


Entropía condicional clase 1 (A1|L1): 1.3115433811024402
Entropía condicional clase 2 (A2|L2): 1.1251274508998534


#### Ahora las condicionales pero entre los anchos $H(A_1 | A_2)$ y los largos $H(L_1 | L_2)$

In [239]:
ancho1 = np.array(ancho_clase1)[:ancho_clase2.shape[0]]
largo1 = np.array(largo_clase1)[:largo_clase2.shape[0]]

hist_anchos, _, _ = np.histogram2d(ancho_clase2, ancho1, bins=5)
joint_prob_anchos = hist_anchos / np.sum(hist_anchos)

hist_largos, _, _ = np.histogram2d(largo_clase2, largo1, bins=5)
joint_prob_largos = hist_largos / np.sum(hist_largos)

print('Entropía conjunta anchos:',joint_entropy(joint_prob_anchos))
print('Entropía conjunta largos:',joint_entropy(joint_prob_largos))

Entropía conjunta anchos: 3.6375530082064356
Entropía conjunta largos: 3.5806979351503374


In [240]:
conditional_prob_anchos = conditional_distribution(joint_prob_anchos)
conditional_prob_largos = conditional_distribution(joint_prob_largos)

print('Entropía condicional anchos (A1|A2):',conditional_entropy(conditional_prob_anchos, joint_prob_anchos))
print('Entropía condicional largos (L1|L2):',conditional_entropy(conditional_prob_largos, joint_prob_largos))


Entropía condicional anchos (A1|A2): 1.8468602935137588
Entropía condicional largos (L1|L2): 1.8522168000712587


#### Información mutua y entropía

$$ I(X;Y) = H(X) - H(X|Y) $$

In [241]:
def mutual_information(marg_prob_X, marg_prob_Y, joint_prob) -> float:
    I = 0
    for x in range(marg_prob_X.shape[0]):
        for y in range(marg_prob_Y.shape[0]):
            if joint_prob[x][y] != 0:
                I += joint_prob[x][y] * np.log2(joint_prob[x][y] / (marg_prob_X[x] * marg_prob_Y[y]))
    return I

hist_clase1, _, _ = np.histogram2d(largo_clase1, ancho_clase1, bins=5)
joint_prob_clase1 = hist_clase1 / np.sum(hist_clase1)

marginal_largo_clase1 = np.sum(joint_prob_clase1, axis=1)
marginal_ancho_clase1 = np.sum(joint_prob_clase1, axis=0)

print('Información mutua clase 1:',mutual_information(marginal_largo_clase1, marginal_ancho_clase1, joint_prob_clase1))


# Lo mismo para ancho1 y ancho2
ancho1 = np.array(ancho_clase1)[:ancho_clase2.shape[0]]
ancho2 = ancho_clase2

hist_anchos, _, _ = np.histogram2d(ancho1, ancho2, bins=5)
joint_prob_anchos = hist_anchos / np.sum(hist_anchos)

marginal_ancho1 = np.sum(joint_prob_anchos, axis=1)
marginal_ancho2 = np.sum(joint_prob_anchos, axis=0)

print('Información mutua anchos:',mutual_information(marginal_ancho1, marginal_ancho2, joint_prob_anchos))


Información mutua clase 1: 0.5300115307683494
Información mutua anchos: 0.0796008638042334


In [242]:
# Corroboración
hist_clase1, _, _ = np.histogram2d(ancho_clase1, largo_clase1, bins=5)
joint_prob_clase1 = hist_clase1 / np.sum(hist_clase1)
conditional_prob_clase1 = conditional_distribution(joint_prob_clase1)
I_clase_1 = entropy(prob_largo_clase1) - conditional_entropy(conditional_prob_clase1, joint_prob_clase1)
print('Información mutua I(L1;A1):',mutual_information(prob_ancho_clase1, prob_largo_clase1, joint_prob_clase1))
print('Información mutua H(L1) - H(L1|A1):', I_clase_1)

hist_clase2, _, _ = np.histogram2d(ancho_clase2, largo_clase2, bins=5)
joint_prob_clase2 = hist_clase2 / np.sum(hist_clase2)
conditional_prob_clase2 = conditional_distribution(joint_prob_clase2)
I_clase_2 = entropy(prob_largo_clase2) - conditional_entropy(conditional_prob_clase2, joint_prob_clase2)
print('Información mutua I(L2;A2):', I_clase_2)
print('Información mutua H(L2) - H(L2|A2):',mutual_information(prob_ancho_clase2, prob_largo_clase2, joint_prob_clase2))


Información mutua I(L1;A1): 0.5300115307683496
Información mutua H(L1) - H(L1|A1): 0.5300115307683493
Información mutua I(L2;A2): 0.6655652637928242
Información mutua H(L2) - H(L2|A2): 0.6655652637928238


#### Entropía relativa e información mutua

In [243]:
def relative_entropy(P: np.ndarray, Q: np.ndarray) -> float:
    D = 0
    for p in range(P.shape[0]):
        for q in range(Q.shape[1]):
            if P[p][q] != 0 and Q[p][q] != 0:
                D += P[p][q] * np.log2(P[p][q] / Q[p][q])
    return D

In [244]:
print('Entropía relativa C1, C2:',relative_entropy(joint_prob_clase1, joint_prob_clase2))

Entropía relativa C1, C2: 0.24180405291993165
